*Even though in this notebook we used astroquery to receive the GAIA designation and the spectral types. there is an easier way. We can directly download the csv file https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=PS from this link. And before downloading, don't forget to select the GAIA ID column.*

### First, we set a query string (col_query) for the columns we are interested in

In [1]:
import requests

columns = ['pl_name','hostname','discoverymethod','disc_refname','pl_orbper','pl_orbpererr1','pl_orbpererr2','pl_orbper_reflink','pl_massj','pl_massjerr1','pl_massjerr2','pl_massjlim','pl_radj','pl_radjerr1','pl_radjerr2','pl_radj_reflink','pl_orbsmax','pl_orbsmaxerr1','pl_orbsmaxerr2','pl_orbsmax_reflink','pl_orbeccen','pl_orbeccenerr1','pl_orbeccenerr2','pl_orbeccen_reflink','pl_orbincl','pl_orbinclerr1','pl_orbinclerr2','pl_orbincl_reflink','pl_orbtper','pl_orbtpererr1','pl_orbtpererr2','pl_orbtper_reflink']
pscompareProb = ['pl_massj', 'pl_massjerr1', 'pl_massjerr2', 'pl_massjlim']

col_query = ''

for i in columns:
    if i not in pscompareProb:
        col_query += "," + i
col_query = col_query[1:]

### Then, we receive the data for all the systems where the number of the stars in the system is 2 and then we convert the received data into a csv file in this directory (we called it nasa_planets.csv)

Note that if you run the following code block, you might get a "permission denied" error because the following nasa_planets.csv is already present in this directory. So either skip the following code block or delete that nasa_planets.csv file and then run the code block

In [5]:
CSV_URL = 'https://exoplanetarchive.ipac.caltech.edu/TAP/sync?query=select+'+str(col_query)+'+from+pscomppars+where+sy_snum+=+2&format=csv'

with open("nasa_planets.csv", 'wb') as f, \
        requests.get(CSV_URL, stream=True) as r:
    for line in r.iter_lines():
        f.write(line+'\n'.encode())

### Now, we will import the csv file we just created and we will manipulate that csv data for our purpose. For example, to receive the name of the systems, we can collect all the unique hostname values.

In [17]:
from pandas import read_fwf, DataFrame, read_csv

In [5]:
df = read_csv("nasa_planets.csv")

In [6]:
df.head()

,pl_name,hostname,discoverymethod,disc_refname,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbper_reflink,pl_radj,pl_radjerr1,...,pl_orbeccenerr2,pl_orbeccen_reflink,pl_orbincl,pl_orbinclerr1,pl_orbinclerr2,pl_orbincl_reflink,pl_orbtper,pl_orbtpererr1,pl_orbtpererr2,pl_orbtper_reflink
0,Kepler-477 b,Kepler-477,Transit,<a refstr=MORTON_ET_AL__2016 href=https://ui.a...,11.119907,1.160000e-05,-1.160000e-05,<a refstr=MORTON_ET_AL__2016 href=https://ui.a...,0.185,0.016,...,NaN,<a refstr=Q1_Q17_DR25_KOI_TABLE href=https://e...,87.750,NaN,NaN,<a refstr=Q1_Q17_DR25_KOI_TABLE href=https://e...,NaN,NaN,NaN,NaN
1,WASP-36 b,WASP-36,Transit,<a refstr=SMITH_ET_AL__2012 href=https://ui.ad...,1.537366,2.400000e-07,-2.400000e-07,<a refstr=MANCINI_ET_AL__2016 href=https://ui....,1.327,0.021,...,NaN,<a refstr=BONOMO_ET_AL__2017 href=https://ui.a...,83.150,0.130,-0.130,<a refstr=MANCINI_ET_AL__2016 href=https://ui....,NaN,NaN,NaN,NaN
2,TOI-451 c,TOI-451,Transit,<a refstr=NEWTON_ET_AL__2021 href=https://ui.a...,9.192522,6.000000e-05,-1.000000e-04,<a refstr=NEWTON_ET_AL__2021 href=https://ui.a...,0.277,0.012,...,NaN,<a refstr=NEWTON_ET_AL__2021 href=https://ui.a...,89.610,0.270,-0.360,<a refstr=NEWTON_ET_AL__2021 href=https://ui.a...,NaN,NaN,NaN,NaN
3,55 Cnc e,55 Cnc,Radial Velocity,<a refstr=MCARTHUR_ET_AL__2004 href=https://ui...,0.736547,1.300000e-06,-1.400000e-06,<a refstr=BOURRIER_ET_AL__2018 href=https://ui...,0.167,0.003,...,-0.03,<a refstr=BOURRIER_ET_AL__2018 href=https://ui...,83.590,0.470,-0.440,<a refstr=BOURRIER_ET_AL__2018 href=https://ui...,2.447578e+06,0.0001,-0.0001,<a refstr=WRIGHT_ET_AL__2009 href=https://ui.a...
4,HAT-P-1 b,HAT-P-1,Transit,<a refstr=BAKOS_ET_AL__2007 href=https://ui.ad...,4.465300,5.500000e-07,-5.500000e-07,<a refstr=NIKOLOV_ET_AL__2014 href=https://ui....,1.319,0.019,...,NaN,<a refstr=MENT_ET_AL__2018 href=https://ui.ads...,85.634,0.056,-0.056,<a refstr=NIKOLOV_ET_AL__2014 href=https://ui....,2.463897e+06,NaN,NaN,<a refstr=MENT_ET_AL__2018 href=https://ui.ads...


### Now we will connect to Simbad database to retrieve the spectral type of the system and add a column called 'Spectral Type' at the end of the dataframe which will contain the spectral type of each host. However, you will notice that many of the hosts are not listed in simbad, so many rows will still be empty.

In [7]:
from astroquery.simbad import Simbad
customSimbad = Simbad() 
customSimbad.add_votable_fields('sptype')

In [8]:
temp_list = []

In [9]:
for i in df['hostname']:
    result = customSimbad.query_object(i)
    try:
        sp_type = result['SP_TYPE'][0]
        temp_list.append(sp_type)
    except:
        temp_list.append('')

C:\Users\muhay\anaconda3\lib\site-packages\astroquery\simbad\core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'Qatar-6' this identifier has an incorrect format for catalog: 	Qatar : Qatar
  warnings.warn("Warning: The script line number %i raised "
C:\Users\muhay\anaconda3\lib\site-packages\astroquery\simbad\core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : KELT-19A
  warnings.warn("Warning: The script line number %i raised "
C:\Users\muhay\anaconda3\lib\site-packages\astroquery\simbad\core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : TOI-1259a
  warnings.warn("Warning: The script line number %i raised "
C:\Users\muhay\anaconda3\lib\site-packages\astroquery\sim

C:\Users\muhay\anaconda3\lib\site-packages\astroquery\simbad\core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'gam 1 Leo': No known catalog could be found
  warnings.warn("Warning: The script line number %i raised "
C:\Users\muhay\anaconda3\lib\site-packages\astroquery\simbad\core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'OGLE-2007-BLG-349L A' this identifier has an incorrect format for catalog: 	OGLE : Optical Gravitational Lensing Event
  warnings.warn("Warning: The script line number %i raised "
C:\Users\muhay\anaconda3\lib\site-packages\astroquery\simbad\core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'TIC 172900988 Aa' this identifier has an incorrect format for catalog: 	TIC : TESS Input Catalog
  warnings.warn("Warning: The 

In [10]:
df['Spectral Type'] = temp_list

### Now we will retrieve the gaia designation for every object. We need gaia designation because we have to query gaia database too and gaia doesn't store the name, only designation. So, we will first retrieve the designation in gaia from simbad, then using that designation we will retrieve ra,dec etc. from gaia

In [21]:
desig_list = []

In [26]:
for i in df['hostname']:
    id_result = Simbad.query_objectids(i)
    try:
        tup = []
        for x in id_result:
            if 'gaia dr2' in x['ID'].lower():
                tup.append(x['ID'])
        desig_list.append(tup)
    except:
        desig_list.append('')

C:\Users\muhay\anaconda3\lib\site-packages\astroquery\simbad\core.py:135: UserWarning: Warning: The script line number 2 raised an error (recorded in the `errors` attribute of the result table): 'Qatar-6' this identifier has an incorrect format for catalog: 	Qatar : Qatar
  warnings.warn("Warning: The script line number %i raised "
C:\Users\muhay\anaconda3\lib\site-packages\astroquery\simbad\core.py:135: UserWarning: Warning: The script line number 2 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : KELT-19A
  warnings.warn("Warning: The script line number %i raised "
C:\Users\muhay\anaconda3\lib\site-packages\astroquery\simbad\core.py:135: UserWarning: Warning: The script line number 2 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : TOI-1259a
  warnings.warn("Warning: The script line number %i raised "
C:\Users\muhay\anaconda3\lib\site-packages\astroquery\sim

In [23]:
df['Gaia Designation'] = desig_list

In [24]:
df.head()

,pl_name,hostname,discoverymethod,disc_refname,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbper_reflink,pl_radj,pl_radjerr1,...,pl_orbincl,pl_orbinclerr1,pl_orbinclerr2,pl_orbincl_reflink,pl_orbtper,pl_orbtpererr1,pl_orbtpererr2,pl_orbtper_reflink,Spectral Type,Gaia Designation
0,Kepler-477 b,Kepler-477,Transit,<a refstr=MORTON_ET_AL__2016 href=https://ui.a...,11.119907,1.160000e-05,-1.160000e-05,<a refstr=MORTON_ET_AL__2016 href=https://ui.a...,0.185,0.016,...,87.750,NaN,NaN,<a refstr=Q1_Q17_DR25_KOI_TABLE href=https://e...,NaN,NaN,NaN,NaN,,[]
1,WASP-36 b,WASP-36,Transit,<a refstr=SMITH_ET_AL__2012 href=https://ui.ad...,1.537366,2.400000e-07,-2.400000e-07,<a refstr=MANCINI_ET_AL__2016 href=https://ui....,1.327,0.021,...,83.150,0.130,-0.130,<a refstr=MANCINI_ET_AL__2016 href=https://ui....,NaN,NaN,NaN,NaN,G2,[Gaia DR2 5750936092375254016]
2,TOI-451 c,TOI-451,Transit,<a refstr=NEWTON_ET_AL__2021 href=https://ui.a...,9.192522,6.000000e-05,-1.000000e-04,<a refstr=NEWTON_ET_AL__2021 href=https://ui.a...,0.277,0.012,...,89.610,0.270,-0.360,<a refstr=NEWTON_ET_AL__2021 href=https://ui.a...,NaN,NaN,NaN,NaN,,[Gaia DR2 4844691297067063424]
3,55 Cnc e,55 Cnc,Radial Velocity,<a refstr=MCARTHUR_ET_AL__2004 href=https://ui...,0.736547,1.300000e-06,-1.400000e-06,<a refstr=BOURRIER_ET_AL__2018 href=https://ui...,0.167,0.003,...,83.590,0.470,-0.440,<a refstr=BOURRIER_ET_AL__2018 href=https://ui...,2.447578e+06,0.0001,-0.0001,<a refstr=WRIGHT_ET_AL__2009 href=https://ui.a...,K0IV-V,[Gaia DR2 704967037090946688]
4,HAT-P-1 b,HAT-P-1,Transit,<a refstr=BAKOS_ET_AL__2007 href=https://ui.ad...,4.465300,5.500000e-07,-5.500000e-07,<a refstr=NIKOLOV_ET_AL__2014 href=https://ui....,1.319,0.019,...,85.634,0.056,-0.056,<a refstr=NIKOLOV_ET_AL__2014 href=https://ui....,2.463897e+06,NaN,NaN,<a refstr=MENT_ET_AL__2018 href=https://ui.ads...,G0V,[Gaia DR2 1928431764627661440]


### Voila! So, now we have gaia designation for almost all the objects. We can use these designation and write similar queries (just like we did for spectral type) from gaia for ra, dec, their errors etc. and add more columns to this dataframe.

In [ ]:
from astroquery.gaia import Gaia
table_metadata = Gaia.load_table("gaiadr2.gaia_source")

In [ ]:
contuned. 

# Finally, after having all the columns in a single dataframe, we can just seperate the columns we need for system, stars, etc. and make new dataframe which we can save as a csv file

In [20]:
final = df[['hostname', '']].copy()

,pl_name,hostname,discoverymethod,disc_refname,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbper_reflink,pl_radj,pl_radjerr1,...,pl_orbincl,pl_orbinclerr1,pl_orbinclerr2,pl_orbincl_reflink,pl_orbtper,pl_orbtpererr1,pl_orbtpererr2,pl_orbtper_reflink,Spectral Type,Gaia Designation
0,Kepler-477 b,Kepler-477,Transit,<a refstr=MORTON_ET_AL__2016 href=https://ui.a...,11.119907,1.160000e-05,-1.160000e-05,<a refstr=MORTON_ET_AL__2016 href=https://ui.a...,0.185,0.016,...,87.750,NaN,NaN,<a refstr=Q1_Q17_DR25_KOI_TABLE href=https://e...,NaN,NaN,NaN,NaN,,[]
1,WASP-36 b,WASP-36,Transit,<a refstr=SMITH_ET_AL__2012 href=https://ui.ad...,1.537366,2.400000e-07,-2.400000e-07,<a refstr=MANCINI_ET_AL__2016 href=https://ui....,1.327,0.021,...,83.150,0.130,-0.130,<a refstr=MANCINI_ET_AL__2016 href=https://ui....,NaN,NaN,NaN,NaN,G2,"[Gaia DR3 5750936092375254016, Gaia DR2 575093..."
2,TOI-451 c,TOI-451,Transit,<a refstr=NEWTON_ET_AL__2021 href=https://ui.a...,9.192522,6.000000e-05,-1.000000e-04,<a refstr=NEWTON_ET_AL__2021 href=https://ui.a...,0.277,0.012,...,89.610,0.270,-0.360,<a refstr=NEWTON_ET_AL__2021 href=https://ui.a...,NaN,NaN,NaN,NaN,,"[Gaia DR3 4844691297067063424, Gaia DR1 484469..."
3,55 Cnc e,55 Cnc,Radial Velocity,<a refstr=MCARTHUR_ET_AL__2004 href=https://ui...,0.736547,1.300000e-06,-1.400000e-06,<a refstr=BOURRIER_ET_AL__2018 href=https://ui...,0.167,0.003,...,83.590,0.470,-0.440,<a refstr=BOURRIER_ET_AL__2018 href=https://ui...,2.447578e+06,0.0001,-0.0001,<a refstr=WRIGHT_ET_AL__2009 href=https://ui.a...,K0IV-V,"[Gaia DR3 704967037090946688, Gaia DR2 7049670..."
4,HAT-P-1 b,HAT-P-1,Transit,<a refstr=BAKOS_ET_AL__2007 href=https://ui.ad...,4.465300,5.500000e-07,-5.500000e-07,<a refstr=NIKOLOV_ET_AL__2014 href=https://ui....,1.319,0.019,...,85.634,0.056,-0.056,<a refstr=NIKOLOV_ET_AL__2014 href=https://ui....,2.463897e+06,NaN,NaN,<a refstr=MENT_ET_AL__2018 href=https://ui.ads...,G0V,"[Gaia DR3 1928431764627661440, Gaia DR2 192843..."


In [18]:
final.to_csv("test.csv", sep=',', encoding='utf-8')